# Q# Interop with OpenQASM

The modern QDK provides interoperability with OpenQASM 3 programs built upon the core Q# compiler infrastructure.

This core enables integration and local resource estimation without relying on external tools. Users are able to estimate resources for their OpenQASM programs locally (see the [resource estimation with Qiskit sample notebook](../../estimation/estimation-openqasm.ipynb)), leveraging the Q# compiler's capabilities for analysis, transformation, code generation, and simulation. This also enables the generation of QIR from OpenQASM progams leveraging the [modern QDKs advanced code generation capabilities](https://devblogs.microsoft.com/qsharp/integrated-hybrid-support-in-the-azure-quantum-development-kit/).

This includes support for classical instructions available in OpenQASM such as for loops, if statements, switch statements, while loops, binary expresssions, and more.

## Running Qiskit circuits
The `QSharpBackend` backend is the main class to interact with for running circuits and generating QIR.

To start, we'll set up a simple circuit with a prepared state.

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(2, 2)
circuit.name = "state_prep"

# State vector to initialize: |ψ⟩ = (|0⟩ - |1⟩) / √2
circuit.initialize([1 / np.sqrt(2), -1 / np.sqrt(2)], 0)
circuit.h(0)
circuit.measure(0, 0)

circuit.prepare_state([1 / np.sqrt(2), -1 / np.sqrt(2)], 1)
circuit.h(1)
circuit.measure(1, 1)

circuit.draw(output="text")

With the circuit created, we can run the circuit with Q#'s backend. By default, it will use the `Unrestricted` profile meaning anything is allowed for simulation.

In [ ]:
from qsharp.interop.qiskit import QSharpBackend

backend = QSharpBackend()
job = backend.run(circuit)
counts = job.result().get_counts()
print(counts)

## Parameterized Qiskit circuits

Some circuits require parameters as input. To start, we'll define utility functions to create parameterized circuit(s).

In [ ]:
from typing import List

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter


def get_theta_range(samples: int) -> List[float]:
    return np.linspace(0, 2 * np.pi, samples)


def get_parameterized_circuit(n: int) -> QuantumCircuit:
    theta = Parameter("θ")
    n = 5
    qc = QuantumCircuit(n, 1)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    qc.barrier()
    qc.rz(theta, range(n))
    qc.barrier()

    for i in reversed(range(n - 1)):
        qc.cx(i, i + 1)
    qc.h(0)
    qc.measure(0, 0)
    return qc


def get_parameterized_circuits(n: int, theta_range: List[float]) -> List[QuantumCircuit]:
    qc = get_parameterized_circuit(n)
    qc.draw()
    theta = qc.parameters[0]
    circuits = [qc.assign_parameters({theta: theta_val}) for theta_val in theta_range]
    return circuits

Attempting to run without binding all input will generate an error in the job.

In [ ]:
from qsharp import QSharpError, TargetProfile
from qsharp.interop.qiskit import QSharpBackend

circuit = get_parameterized_circuit(3)
backend = QSharpBackend()
try:
    backend.qir(circuit, target_profile=TargetProfile.Base)
except QSharpError as e:
    print(e)

Any parameters must be bound before we can run the circuit. As we can see from the exception output, we must define the value for the input parameter `θ`. To do this, set the `params` argument to the `run` function.

In [ ]:
from qsharp.interop.qiskit import QSharpBackend

circuit = get_parameterized_circuit(3)
backend = QSharpBackend()

circuit.assign_parameters(
    {"θ": "0.5"},
    inplace=True,
)
job = backend.run(circuit)
counts = job.result().get_counts()
print(counts)

## Classical instructions in circuits

### Run Qiskit with classical instructions
Qiskit has begun implementing some classical computation support as they expand their OpenQASM 3 support. These constructs, insofar as Qiskit can export them, can be consumed by Q#.

As an example, we can create a classical switch statement in Qiskit and run the program.

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit.circuit import (
    QuantumCircuit,
)

from qsharp import QSharpError, TargetProfile

qreg = QuantumRegister(3, name="q")
creg = ClassicalRegister(3, name="c")
qc = QuantumCircuit(qreg, creg)
qc.h([0, 1, 2])
qc.measure_all(add_bits=False)

with qc.switch(creg) as case:
    with case(7):
        qc.x(0)
    with case(1, 2):
        qc.z(1)
    with case(case.DEFAULT):
        qc.cx(0, 1)
qc.measure_all(add_bits=False)

backend = QSharpBackend()

print(backend.run(qc).result())

Using that same circuit, we can generate QIR which is used to run on quantum hardware.

In [ ]:
backend = QSharpBackend(target_profile=TargetProfile.Adaptive_RI)
print(backend.qir(qc))
from qsharp.interop.qasm3 import eval, estimate, import_callable

Not all programs can run on all hardware. Here we can try to target the `Base` profile, but we will get detailed errors on which parts of the program aren't supported.

In [ ]:
try:
    backend.qir(qc, target_profile=TargetProfile.Base)
except QSharpError as e:
    print(e)

## Errors
#### Unsupported language features, `QasmError`, and `QSharpError`
The modern QDK's interop with Qiskit is based on Qiskit's OpenQASM 3 support. Qiskit supports a subset of OpenQASM 3 features which may cause issues during conversion.

If the Qiskit OpenQASM `Exporter` or OpenQASM parser don't support the feature yet, a `QasmError` is raised prior to conversion. When an OpenQASM parsing failure occurs, this is likely an issue with the Qiskit libraries parsing and/or export functionality. Additionally, failure to transform the OpenQASM into Q#'s internal representation will throw a `QasmError`. This is most likely due to a semantically invalid OpenQASM program as input or an unsupported language feauture is being used.

If the program can't be compiled to QIR, has invalid input bindings, or encounters a runtime error, a `QSharpError` is raised.

If the backend configuration is not valid for a given operation, a `ValueError` may be raised. This is most likely caused by trying to generate QIR with the `Unrestricted` profile.

### Semantic Errors
It is still possible to create circuits that are semantically invalid. These will raise `QasmErrors` as the OpenQASM can't be compiled.


We'll look at examples of each scenario.


#### General semantic errors
Most common semantic errors arise from unsupported features:
- No classical registers defined. If the circuit(s) being used do not measure into classical registers then the circuit is purely classical.
- Aliases were used for classical registers.

Example, creating a circuit without any output:

In [ ]:
from qsharp.interop.qiskit import QasmError

try:
    circuit = QuantumCircuit(2)
    circuit.x(0)
    backend = QSharpBackend()
    print(backend.run(circuit).result())
except QasmError as ex:
    print(ex)

Example, using aliased classical registers:

In [ ]:
from qsharp.interop.qiskit import QasmError

try:
    q = QuantumRegister(2, name="q")
    cr1 = ClassicalRegister(1, name="cr1")
    cr2 = ClassicalRegister(1, name="cr2")
    # Create a ClassicalRegister with bits from two different QuantumRegisters
    # which is not supported by the Q# backend.
    cr3 = ClassicalRegister(
        name="cr3",
        bits=[
            cr1[0],
            cr2[0],
        ],
    )
    qc = QuantumCircuit(q, cr1, cr2, cr3)

    backend = QSharpBackend(target_profile=TargetProfile.Base)
    backend.qir(qc)
except QasmError as ex:
    print(f"Exception: {str(ex)}")
    # Print the cause of the exception if it exists.
    # This will print the error message from Qiskit itself.
    if ex.__cause__:
        print(f"Cause: {str(ex.__cause__)}")

#### QIR generation semantic errors

When targetting harware by compiling to QIR there are additional restrictions which may cause compilation errors. Most common scenarios:
- Trying to generate QIR when the profile is set to `Unrestricted`. `Unrestricted` is only valid for simulation. Either `TargetProfile.Base` or `TargetProfile.Adaptive_RI` must be used.
- Not all bits in classical registers have been assigned to. Usually because there were no measurements, or extra registers were declared.



Example, generating QIR with `Unrestricted`

In [ ]:
try:
    circuit = QuantumCircuit(1)
    circuit.x(0)
    circuit.measure_all()
    backend = QSharpBackend()
    print(backend.qir(circuit))
except ValueError as ex:
    print(ex)

To avoid this issue, set the `target_profile` argument either in the `QSharpBackend` creation or in the `backend.qir` call.

When generating `QIR`, all output registers must be read into before generating QIR. Failure to do so results in a `QSharpError`.

In this next example, we declare two output bits, but only measure into one. This causes an error because result values can only be a side effect of measurement, and cannot be used like classical variables when compiling for hardware.

In [ ]:
circuit = QuantumCircuit(2, 2)
circuit.x(0)
circuit.measure(0, 1)
backend = QSharpBackend(target_profile=TargetProfile.Base)
try:
    print(backend.qir(circuit))
except QSharpError as ex:
    print(ex)

In [ ]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

In [ ]:
%%qasm3 --name bell
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
bit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];

In [ ]:
from qsharp.code import bell
bell()

In [ ]:
%%qsharp
bell()

In [ ]:
from qsharp_widgets import Histogram

Histogram(qsharp.run("bell()", shots=1000, noise=qsharp.DepolarizingNoise(0.01)))

In [ ]:
#qsharp.circuit(qsharp.code.bell2, 1.57)
import qsharp.code
from qsharp_widgets import Circuit
qsharp.circuit(qsharp.code.bell)


In [ ]:
from qsharp_widgets import Circuit
Circuit(qsharp.circuit(qsharp.code.bell))

In [ ]:
print(qsharp.compile(bell))

In [ ]:
%%qasm3 --name needs_args
include "stdgates.inc";
input float theta;
qubit[2] q;
rx(theta) q[0];
rx(-theta) q[1];
bit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];

In [ ]:
from qsharp.code import needs_args
print(qsharp.compile(needs_args, 1.57))

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
#    !pip install --upgrade pip
    !pip install cirq
    import cirq

    print("installed cirq.")

In [ ]:
import random
import cirq

def make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit):
    """Gates implementing the function f(a) = a·factors + bias (mod 2)."""

    if secret_bias_bit:
        yield cirq.X(output_qubit)

    for qubit, bit in zip(input_qubits, secret_factor_bits):
        if bit:  # pragma: no cover
            yield cirq.CNOT(qubit, output_qubit)


def make_bernstein_vazirani_circuit_with_cirq(input_qubits, output_qubit, oracle):
    """Solves for factors in f(a) = a·factors + bias (mod 2) with one query."""

    c = cirq.Circuit()

    # Initialize qubits.
    c.append([cirq.X(output_qubit), cirq.H(output_qubit), cirq.H.on_each(*input_qubits)])

    # Query oracle.
    c.append(oracle)

    # Measure in X basis.
    c.append([cirq.H.on_each(*input_qubits), cirq.measure(*input_qubits, key='result')])

    return c


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

# Embed the oracle into a special quantum circuit querying it exactly once.
#circuit = make_bernstein_vazirani_circuit_with_cirq(input_qubits, output_qubit, oracle)
#print('Circuit:')
#print(circuit)

In [ ]:
qasm_str = cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0"))
#qasm_str = circuit.to_qasm("3.0")
print(qasm_str)
#cirq.__version__

In [ ]:
import qsharp
from qsharp.interop.qasm3 import eval, estimate, import_callable
from qsharp import init
init(target_profile=qsharp.TargetProfile.Base)

qubit_count=8

# Choose qubits to use.
input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
output_qubit = cirq.GridQubit(qubit_count, 0)

# Pick coefficients for the oracle and create a circuit to query it.
secret_bias_bit = random.randint(0, 1)
secret_factor_bits = [random.randint(0, 1) for _ in range(qubit_count)]
oracle = make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit)
print(
    'Secret function:\nf(a) = '
    f"a·<{', '.join(str(e) for e in secret_factor_bits)}> + "
    f"{secret_bias_bit} (mod 2)"
)

circuit = make_bernstein_vazirani_circuit_with_cirq(input_qubits, output_qubit, oracle)

# use QDK resource estimation on cirq circuit via QASM3
estimate(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))

In [ ]:
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.measure(q0, key='a'),
    cirq.H(q1).with_classical_controls('a'),
    cirq.measure(q1, key='b'),
)
print(circuit)
print(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))

In [ ]:
alice = cirq.NamedQubit('alice')
bob = cirq.NamedQubit('bob')
message = cirq.NamedQubit('_message')

message_circuit = cirq.Circuit(
    # Create the message.
    cirq.X(message) ** 0.371,
    cirq.Y(message) ** 0.882,
)

circuit = cirq.Circuit(
    # Create Bell state to be shared between Alice and Bob.
    cirq.H(alice),
    cirq.CNOT(alice, bob),
    # Prepare message circuit
    message_circuit,
    # Bell measurement of the message and Alice's entangled qubit.
    cirq.CNOT(message, alice),
    cirq.H(message),
    cirq.measure(message, key='M'),
    cirq.measure(alice, key='A'),
    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum message on Bob's entangled qubit.
    cirq.X(bob).with_classical_controls('A'),
    cirq.Z(bob).with_classical_controls('M'),
)
print(circuit)
print(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))

In [ ]:
import cirq
from qsharp.interop.qasm3 import estimate

q0 = cirq.LineQubit(0)
subcircuit = cirq.FrozenCircuit(cirq.measure(q0, key='a'), cirq.X(q0).with_classical_controls('a'))
circuit = cirq.Circuit(
    cirq.measure(q0, key='a'),
    cirq.CircuitOperation(subcircuit, repetitions=2),
    cirq.X(q0).with_classical_controls('a'),
)
print(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))
estimate(cirq.qasm(circuit, args=cirq.QasmArgs(version="3.0")))